In [1]:
import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
import datetime
import time
import csv
import json
import nltk
import re
from pprint import pprint

from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
import re

Obteniendo los datos de Twitter (1000 registros)

In [2]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [3]:
def search(hashtag, Max_Tweets, Recdata):

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    
    api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    data = Cursor(api.search, q=hashtag, lang='en').items(Max_Tweets)
    
    current_working_dir = "./"
    log_tweets = current_working_dir  + 'JSONtweets.json'
    with open(log_tweets, 'w') as outfile:
        for tweet in data:
            Recdata.append(json.loads(json.dumps(tweet._json)))
            outfile.write(json.dumps(tweet._json))
            outfile.write("\n")
            
    # Podemos Crear dataset con esta informacion
    tweets = pd.DataFrame()
    # Campos del Tweet
    tweets['created_at'] = map(lambda tweet: time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(tweet['created_at'],'%a %b %d %H:%M:%S +0000 %Y')), Recdata)
    tweets['user'] = map(lambda tweet: tweet['user']['screen_name'], Recdata)
    tweets['text'] = map(lambda tweet: tweet['text'].encode('utf-8'), Recdata)
   
    # Generar un archivo cvs
    tweets.to_csv('CVStweets.csv', sep=';', header=False)

In [4]:
#Max_Tweets = 1000
#tweets_data = []
#hashtag = "#USAvCRC"

#search(hashtag, Max_Tweets, tweets_data)

Cargamos los datos que guardamos en el archivo de Tweet, para no ir siempre a Twitter, aunque el codigo utilizado para la descarga de tweets, esta al inicio de esta practica.

Primero cargaremos la lista de: ** Palabas Positivos, Negativas e Insultos **

In [5]:
positive_list = []
text_file = open('positive-words.txt', 'r')
positive_list = text_file.read().split('\n')

In [6]:
negative_list = []
text_file = open('negative-words.txt', 'r')
negative_list = text_file.read().split('\n')

In [7]:
badwords_list = []
text_file = open('bad-words.txt', 'r')
badwords_list = text_file.read().split('\n')

Ahora cargamos los Tweets

In [8]:
tweet_list = []
with open('CVStweets.csv', 'r') as f:
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        reg, tw_date, tw_user, tw_text = row
        tweet_list.append(tw_text)

Tokenizamos la lista de tweets y creamos un nuevo lista

In [9]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [10]:
tweet_token = []
for line in tweet_list:
    tweet_token.append(preprocess(line))

Limpiaremos los tweets de las comillas ("")

In [30]:
# define punctuation
Classified_tweetsNP = []
punctuations = '''';\n"'''

for i in range(len(tweet_list)):
    my_str = tweet_list[i]
    # To take input from the user
    # my_str = input("Enter a string: ")
    
    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char
    Classified_tweetsNP.append(no_punct)

Clasificamos los Tweets en: ** Positivos, Negativos, Neutro y Con insultos **

In [31]:
Classified_tweets = []
for i in range(len(tweet_token)):
    tweetTK_review = tweet_token[i]
    score = 0
    insult = 0
    for tw_token in tweetTK_review:
        if tw_token.strip() in badwords_list:
            insult +=1
        if tw_token.strip() in positive_list:
            score +=1
        if tw_token.strip() in negative_list:
            score +=1      
    # Creamos una nueva lista clasificada
    if insult > 0:
        fwtweet = '\'' + Classified_tweetsNP[i] + '\';\'' + 'Insult' + '\''
#        Classified_tweets_ins.append(fwtweet)
    elif score > 0:
        fwtweet = '\'' + Classified_tweetsNP[i] + '\';\'' + 'Positive' + '\''
#        Classified_tweets_pos.append(fwtweet)
    elif score < 0:
        fwtweet = '\'' + Classified_tweetsNP[i] + '\';\'' + 'Negative' + '\''
#        Classified_tweets_neg.append(fwtweet)
    else:
        fwtweet = '\'' + Classified_tweetsNP[i] + '\';\'' + 'Neutral' + '\''
#        Classified_tweets_neu.append(fwtweet)
    Classified_tweets.append(fwtweet)

#    print(tweet_list[i])
#    print(tw_token)
#    print(score)
#    print(insult)
#    print('\n')

Creamos archivos con las nuevas listas con los tweets clasificados

In [32]:
np.savetxt("Classified_tweets.csv", Classified_tweets,fmt='%s')

In [33]:
negative = []
positive = []

cont = 0
with open('Classified_tweets.csv', 'r') as f:
    reader = csv.reader(f, delimiter=';')
    for row in reader:
        cont = cont + 1
        print(cont)
        print(row)
        tweet, polarity = row
        if polarity == 'Positive':
            positive.append((tweet, 'positive'))
        elif polarity == 'Negative':
            negative.append((tweet, 'negative'))

1
["'RT @ussoccer: One more for Costa Rica. #USAvCRC, 0-4, 78'", "'Neutral'"]
2
["'GET 85 QUALITY FOLLOWERS  https://t.co/dRPo0XxzD9  #Bolivia #VamosChileCTM #USAvCRC #TimeTunnel #followers'", "'Neutral'"]
3
["'RT @TayKZY: Juergen klinsmans ass is getting crisp and toasty on that hot seat #usmnt #USAvCRC #hottake #hotseattake #HotSeatMissIndonesia'", "'Insult'"]
4
["'RT @WrldSoccerShop: GIVEAWAY! If the #USMNT win the #USAvCRC match, well give away a #USA jersey. Follow and RT to enter! https://t.co/bKm\xe2\x80\xa6'", "'Positive'"]
5
["'RT @AlexiLalas: I miss the old Saprissa. #USAvCRC'", "'Positive'"]
6
["'RT @Blacksburke: Today and this week, we all are Brad Guzans face.  #Pray4Guzan #InTheFace #USAvCRC @MenInBlazers https://t.co/EGnUnqbrHX'", "'Neutral'"]
7
["'RT @SoccerInsider: Costa Ricas Canadian-based MLS players are killing the U.S. overseas defenders. #USAvCRC'", "'Positive'"]
8
["'RT @JAShock: Biden: It could always be worseObama: Joe, pleaseBiden: It coulda been 5-0 #USAvCRC